<a href="https://colab.research.google.com/github/ajsanjoaquin/COVID-19-Scanner/blob/master/covidtesting_notebook(Resnet34).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This model is provided as-is, and not meant to diagnose  COVID-19. This model has no clinical approval, nor endorsements from any health organizations. At the moment, this model is a proof of concept. In no way is the author responsible for any damages resulting from using this model. License: MIT 

#PURE PYTORCH IMPLEMENTATION

Model instantiation

In [0]:
#download model
!wget -O corona_resnet34.pth https://www.dropbox.com/s/o27w0dik8hdjaab/corona_resnet34.pth?dl=0

In [0]:
#Code taken from Andrea de Luca (https://bit.ly/2YXW6xN)
## The code below gives you Flatten and the double Adaptive Pooling (from fastai), plus
## a viable head. You must fill the number of FC's nodes manually through the myhead function
from torch import Tensor
from torch import nn
from torchvision import transforms
from torch.autograd import Variable
import PIL.Image
import torch
import torchvision
import logging as log
from typing import Optional # required for "Optional[type]"
import os,re
#CPU for inference
device = torch.device("cpu")

#put test images in test folder
if not os.path.isdir('test'):
  os.makedirs('test')

class Flatten(nn.Module):
    "Flatten `x` to a single dimension, often used at the end of a model. `full` for rank-1 tensor"
    def __init__(self, full:bool=False):
        super().__init__()
        self.full = full

    def forward(self, x):
        return x.view(-1) if self.full else x.view(x.size(0), -1)

class AdaptiveConcatPool2d(nn.Module):
    "Layer that concats `AdaptiveAvgPool2d` and `AdaptiveMaxPool2d`." # from pytorch
    def __init__(self, sz:Optional[int]=None): 
        "Output will be 2*sz or 2 if sz is None"
        super().__init__()
        self.output_size = sz or 1
        self.ap = nn.AdaptiveAvgPool2d(self.output_size)
        self.mp = nn.AdaptiveMaxPool2d(self.output_size)
    def forward(self, x): return torch.cat([self.mp(x), self.ap(x)], 1)
    
def myhead(nf, nc):
    '''
    Inputs: nf=  # of in_features in the 4th layer , nc= # of classes
    '''
    return \
    nn.Sequential(        # the dropout is needed otherwise you cannot load the weights
            AdaptiveConcatPool2d(),
            Flatten(),
            nn.BatchNorm1d(nf,eps=1e-05,momentum=0.1,affine=True,track_running_stats=True),
            nn.Dropout(p=0.25,inplace=False),
            nn.Linear(nf, 512,bias=True),
            nn.ReLU(inplace=True),
            nn.BatchNorm1d(512,eps=1e-05,momentum=0.1,affine=True,track_running_stats=True),
            nn.Dropout(p=0.5,inplace=False),
            nn.Linear(512, nc,bias=True),
        )


my_model=torchvision.models.resnet34() 
modules=list(my_model.children())
modules.pop(-1) 
modules.pop(-1) 
temp=nn.Sequential(nn.Sequential(*modules))
tempchildren=list(temp.children()) 

#append the special fastai head
#Configured according to Model Architecture

tempchildren.append(myhead(1024,3))
model_r34=nn.Sequential(*tempchildren)

#LOAD MODEL
state = torch.load('corona_resnet34.pth')
model_r34.load_state_dict(state['model'])


#important to set to evaluation mode
model_r34.eval()


test_transforms = transforms.Compose([
    transforms.Resize(512),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225])
])

def predict_image(model,image):
    softmaxer = torch.nn.Softmax(dim=1)
    image_tensor = PIL.Image.open(image)
    image_tensor = image_tensor.convert('RGB')
    image_tensor = test_transforms(image_tensor).float()
    image_tensor=image_tensor.unsqueeze(0)

    #convert evaluation to probabilities with softmax
    with torch.no_grad(): #turn off backpropagation
      processed=softmaxer(model(image_tensor))
    return processed[0] #return probabilities


##Before running the code below, put the test images in the test folder that was just created.

In [0]:
path_to_test='test/'
test_files=[file for file in sorted(os.listdir(path_to_test))]
pytorch_results={filename:predict_image(model_r34,path_to_test+filename) for filename in test_files}


###Results are saved in a .csv file in the colab workspace.

In [0]:
import pandas as pd
final_df=pd.DataFrame.from_dict(pytorch_results,orient='index',columns=['covid','nofinding','opacity']).rename_axis('filename').reset_index()
final_df['covid']=final_df['covid'].apply(lambda x: x.item())
final_df['nofinding']=final_df['nofinding'].apply(lambda x: x.item())
final_df['opacity']=final_df['opacity'].apply(lambda x: x.item())
#get the column name of the highest probability
final_df['Predicted Label'] =final_df[['covid','opacity','nofinding']].idxmax(axis=1)
final_df.to_csv('results.csv', header=True)